<a href="https://colab.research.google.com/github/lizhjort/Insight/blob/master/Final_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import matplotlib.pyplot as plt

In [0]:
inventory = pd.read_pickle('/content/gdrive/My Drive/insight_data/min_inventory.pkl.gz')

In [0]:
inventory.head()

,BibNumber,Title,Author,ISBN,PublicationYear,Publisher,ReportDate,ItemCount,MonthYear
44,3259688,The English daughter / Maggie Wadey.,"Wadey, Maggie","1910985139, 9781910985137",2016.,"Sandstone Press Ltd,",2017-09-01,2,2017-09
58,2130621,Fire fighters : stories of survival from the f...,NaN,1560254025,c2002.,Thunder's Mouth Press : Distributed by Publish...,2017-09-01,2,2017-09
62,3068849,The autumn republic / Brian McClellan.,"McClellan, Brian, 1986-","0316219118, 0316219126, 9780316219112, 9780316...",2015.,"Orbit,",2017-09-01,2,2017-09
63,2044450,"American furniture : tables, chairs, sofas & b...","Schwartz, Marvin D.",157912108X,c2000.,Black Dog & Leventhal : Distributed by Workman...,2017-09-01,2,2017-09
75,3165152,No shred of evidence / Charles Todd.,"Todd, Charles","0062386182, 9780062386182",[2016],"William Morrow, an imprint of HarperCollinsPub...",2017-09-01,4,2017-09


In [0]:
results_new = pd.read_pickle('/content/gdrive/My Drive/insight_data/results_new.pkl.gz')

In [0]:
unique_books = results_new['BibNumber'].unique()
unique_books = pd.DataFrame(data=unique_books)
unique_books.shape

(80869, 1)

In [0]:
results_dropped = results_new.drop_duplicates(subset=['BibNumber', 'CheckoutDate'])

In [0]:
results_dropped.shape

(403866, 7)

In [0]:
copies = results_dropped.loc[results_new['BibNumber'] == 3400549]

In [0]:
copies

,BibNumber,CheckoutDate,CheckoutCountNextWeek,Title,ISBN,Predictions,MonthYear
491485,3400549,2019-05-05,403.0,Where the crawdads sing / Delia Owens.,"1984827618, 9781984827616",273.546545,2019-05
491487,3400549,2019-05-12,465.0,Where the crawdads sing / Delia Owens.,"1984827618, 9781984827616",294.335550,2019-05
491489,3400549,2019-05-19,279.0,Where the crawdads sing / Delia Owens.,"1984827618, 9781984827616",300.384839,2019-05
491491,3400549,2019-05-26,372.0,Where the crawdads sing / Delia Owens.,"1984827618, 9781984827616",274.774859,2019-05
491493,3400549,2019-06-02,186.0,Where the crawdads sing / Delia Owens.,"1984827618, 9781984827616",287.497042,2019-06


In [0]:
predictions_by_week = results_dropped.groupby(by=['BibNumber', 'Title', 'ISBN', 'CheckoutDate'])['Predictions'].sum()

In [0]:
predictions_by_week.head()

BibNumber  Title                           ISBN                       CheckoutDate
10143      The price / by Arthur Miller..  082220911X, 9780822209119  2019-05-05      0.0
                                                                      2019-05-12      0.0
                                                                      2019-05-19      0.0
                                                                      2019-05-26      0.0
                                                                      2019-06-02      0.0
Name: Predictions, dtype: float64

In [0]:
sorted_predictions_week = predictions_by_week.sort_values(ascending=False)

In [0]:
sorted_predictions_week.head(25)

BibNumber  Title                                                                                                                                              ISBN                                                  CheckoutDate
3214250    Scythe / Neal Shusterman.                                                                                                                          1442472421, 144247243X, 9781442472426, 9781442472433  2019-05-19      346.107477
2875198    Eleanor & Park / Rainbow Rowell.                                                                                                                   1250012570, 9781250012579                             2019-05-26      345.882663
3215998    Norse mythology / Neil Gaiman.                                                                                                                     039360909X, 1408886804, 9780393609097, 9781408886809  2019-06-02      345.230438
2968772    All the light we cannot see : a novel / Anthony

In [0]:
predictions_by_month = results_dropped.groupby(by=['BibNumber', 'Title', 'ISBN'])['Predictions'].sum()

In [0]:
predictions_by_month.head()

BibNumber  Title                                                                                                                           ISBN                                                                                                                                                                                                                                                                                          
10143      The price / by Arthur Miller..                                                                                                  082220911X, 9780822209119                                                                                                                                                                                                                                                                         0.0
25043      The Cambridge ancient history.                                                                                                  05

In [0]:
sorted_predictions = predictions_by_month.sort_values(ascending=False)

In [0]:
sorted_predictions.head(25)

BibNumber  Title                                                                                                                                                                                                                                                                                                                                                    ISBN                                                                                                      
3305635    Why we sleep : unlocking the power of sleep and dreams / Matthew Walker.                                                                                                                                                                                                                                                                                 1501144316, 1501144324, 9781501144318, 9781501144325                                                          1560.958995
3238160    Hunger : a memoir of (my) body / Roxane Gay.    

In [0]:
sorted_predictions = sorted_predictions.reset_index()

In [0]:
sorted_predictions['Predictions'] = (sorted_predictions['Predictions'] * 0.3)

In [0]:
total_inventory = inventory.groupby(by=['BibNumber', 'Title', 'ISBN', 'ReportDate'])['ItemCount'].sum()

In [0]:
total_inventory = total_inventory.reset_index()

In [0]:
total_inventory

,BibNumber,Title,ISBN,ReportDate,ItemCount
0,10143,The price / by Arthur Miller..,"082220911X, 9780822209119",2017-09-01,3
1,10143,The price / by Arthur Miller..,"082220911X, 9780822209119",2017-10-01,3
2,10143,The price / by Arthur Miller..,"082220911X, 9780822209119",2017-11-01,3
3,10143,The price / by Arthur Miller..,"082220911X, 9780822209119",2017-12-01,3
4,10143,The price / by Arthur Miller..,"082220911X, 9780822209119",2018-01-01,3
5,10143,The price / by Arthur Miller..,"082220911X, 9780822209119",2018-02-01,3
6,10143,The price / by Arthur Miller..,"082220911X, 9780822209119",2018-03-01,3
7,10143,The price / by Arthur Miller..,"082220911X, 9780822209119",2018-04-01,3
8,10143,The price / by Arthur Miller..,"082220911X, 9780822209119",2018-05-01,3
9,10143,The price / by Arthur Miller..,"082220911X, 9780822209119",2018-06-01,3


In [0]:
may_inventory = total_inventory.loc[total_inventory['ReportDate'] == '2019-05-01']


In [0]:
may_inventory.head()

,BibNumber,Title,ISBN,ReportDate,ItemCount
20,10143,The price / by Arthur Miller..,"082220911X, 9780822209119",2019-05-01,3
41,25043,The Cambridge ancient history.,"0521070511, 0521077915, 0521082307, 0521086914...",2019-05-01,2
81,386573,"Lenin, a political life / Robert Service.","0253333245, 0253333253, 0253351812",2019-05-01,2
102,399353,"Necessary losses : the loves, illusions, depen...","0684844958, 9780684844954",2019-05-01,2
123,401655,The sportswriter / Richard Ford.,"0394743253, 0679762108",2019-05-01,2


In [0]:
may = pd.read_pickle('/content/gdrive/My Drive/insight_data/may_inv.pkl.gz')

In [0]:
may

,Title,BibNumber,ItemCount
0,Becoming / Michelle Obama.,133484388,553
1,Where the crawdads sing / Delia Owens.,145314937,382
2,Educated : a memoir / Tara Westover.,102853162,357
3,Unsheltered : a novel / Barbara Kingsolver.,104636377,310
4,Normal people : a novel / Sally Rooney.,106247974,303
5,Nine perfect strangers / Liane Moriarty.,119726921,288
6,The library book / Susan Orlean.,111338840,278
7,Bad blood : secrets and lies in a Silicon Vall...,117811525,274
8,The overstory : a novel / Richard Powers.,100333740,250
9,The best we could do : an illustrated memoir /...,96910050,247


In [0]:
may_merged = pd.merge(sorted_predictions, may, on=['Title'], how='left')

In [0]:
may_merged.sort_values(by='Predictions', ascending=False)

,index,BibNumber_x,Title,ISBN,Predictions,BibNumber_y,ItemCount
0,0,3305635,Why we sleep : unlocking the power of sleep an...,"1501144316, 1501144324, 9781501144318, 9781501...",468.287699,6611270.0,23.0
1,1,3238160,Hunger : a memoir of (my) body / Roxane Gay.,"0062362593, 0062569716, 9780062362599, 9780062...",443.817039,19428960.0,24.0
2,2,3109742,The 7 habits of highly effective people : powe...,"1451639619, 1476740054, 9781451639612, 9781476...",433.798321,18658452.0,17.0
3,3,3028938,"Ms. Marvel. [Vol. 1, No normal] / writer, G. W...","078519021X, 9780785190219",429.358751,12115752.0,9.0
4,4,3400549,Where the crawdads sing / Delia Owens.,"1984827618, 9781984827616",429.161651,145314937.0,382.0
5,5,2687445,If Beale Street could talk : a novel / James B...,"0307275930, 9780307275936",426.704993,8062335.0,16.0
6,6,3194058,"The hidden life of trees : what they feel, how...","1771642483, 9781771642484",419.700172,6388116.0,16.0
7,7,2614624,The immortal life of Henrietta Lacks / Rebecca...,"1400052173, 9781400052172",418.276057,5229248.0,5.0
8,8,2356220,"Bone. [Vol. 3], Eyes of the storm / by Jeff Sm...","0439706254, 9780439706254",416.933099,11781100.0,11.0
9,9,3298319,The girl who takes an eye for an eye / David L...,"0451494326, 9780451494320",414.548948,13193276.0,16.0


In [0]:
may_merged.drop(labels='BibNumber_y', axis=1, inplace=True)

In [0]:
may_merged.head()

,BibNumber_x,Title,ISBN,Predictions,ItemCount
0,3305635,Why we sleep : unlocking the power of sleep an...,"1501144316, 1501144324, 9781501144318, 9781501...",468.287699,23.0
1,3238160,Hunger : a memoir of (my) body / Roxane Gay.,"0062362593, 0062569716, 9780062362599, 9780062...",443.817039,24.0
2,3109742,The 7 habits of highly effective people : powe...,"1451639619, 1476740054, 9781451639612, 9781476...",433.798321,17.0
3,3028938,"Ms. Marvel. [Vol. 1, No normal] / writer, G. W...","078519021X, 9780785190219",429.358751,9.0
4,3400549,Where the crawdads sing / Delia Owens.,"1984827618, 9781984827616",429.161651,382.0


In [0]:
top_book_titles = may_merged

In [0]:
may_merged['Predictions'].sum()

221296.65754379367

In [0]:
may_merged.to_csv('/content/gdrive/My Drive/insight_data/top_books.csv')

In [0]:
reduce_inventory = may_merged.loc[(1.5 * may_merged['ItemCount']) > (may_merged['Predictions'])]

In [0]:
reduce_inventory.shape

(56797, 7)

In [0]:
reduce_inventory['Difference'] = (reduce_inventory['ItemCount'] - reduce_inventory['Predictions'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
reduce_inventory['Difference'].sum()

119194.60324493641

In [0]:

reduce_inventory.sort_values(by='Difference', ascending=False)

,index,BibNumber_x,Title,ISBN,Predictions,BibNumber_y,ItemCount,Difference
702,702,3423722,Becoming / Michelle Obama.,"0525633758, 9780525633754",90.940233,133484388.0,553.0,462.059767
743,743,3362227,Educated : a memoir / Tara Westover.,"0525589988, 9780525589983",86.394305,102853162.0,357.0,270.605695
1099,1099,3420191,The library book / Susan Orlean.,"1432856464, 9781432856465",51.709779,111338840.0,278.0,226.290221
5996,5996,3447100,Gingerbread / Helen Oyeyemi.,"1984882899, 9781984882899",2.422330,106324750.0,213.0,210.577670
10709,10709,3418741,Almost everything : notes on hope / Anne Lamott.,"198482760X, 9781984827609",0.157655,101061450.0,202.0,201.842345
830,830,3376163,Bad blood : secrets and lies in a Silicon Vall...,"1984833634, 9781984833631",75.704492,117811525.0,274.0,198.295508
12151,12151,3447089,The river : a novel / Peter Heller.,"1984883267, 9781984883261",0.063924,99192613.0,196.0,195.936076
499,499,3420712,Nine perfect strangers / Liane Moriarty.,"1432859005, 9781432859008",115.274112,119726921.0,288.0,172.725888
41739,41739,2833273,The World Book encyclopedia.,"0716601133, 9780716601135",0.000000,28116283.0,170.0,170.000000
39356,39356,2528613,The World Book encyclopedia.,"0716601095, 9780716601098",0.000000,28116283.0,170.0,170.000000


In [0]:
reduce_inventory.to_csv('/content/gdrive/My Drive/insight_data/reduce_titles.csv')

In [0]:
reduce_list = reduce_inventory.drop([''])